<a href="https://colab.research.google.com/github/Idk507/AI-Cure-Where-AI-Meets-Healing-Touch/blob/main/LOTR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import chromadb
from langchain.vectorstores import Chroma
from langchain.document_transformers import LongContextReorder
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import  HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers.merger_retriever import MergerRetriever


In [3]:
!pip install chromadb langchain langchain_community openai tiktoken accelerate transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.3 MB/s eta 0:00:0

In [8]:
!pip install sentence_transformers huggingface huggingface_hub pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 8.7 MB/s eta 0:00:00


In [6]:
model_name = "BAAI/bge-large-en"
model_kwargs = {'device':'cpu'}

encode_kwargs = {'normalize_embeddings':False}

hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [9]:
#Data Preprocessing

loader_un_sdg = PyPDFLoader(file_path='/content/RAG_NLP.pdf')
document_un_sdg = loader_un_sdg.load()
text_splitter_un_sdg = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs_un_sdg = text_splitter_un_sdg.split_documents(document_un_sdg)


In [13]:
#Data Preprocessing

loader_paris = PyPDFLoader(file_path='/content/2312.10997v5.pdf')
document_paris  = loader_un_sdg.load()
text_splitter_pars = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs_paris = text_splitter_un_sdg.split_documents(document_un_sdg)


In [ ]:
#create and store vectors

un_sdg_db = Chroma.from_documents(docs_un_sdg, hf,collection_metadata={"hnsw:space": "cosine"},persist_directory="/content/un_sdg_db")
paris_db = Chroma.from_documents(docs_paris, hf,collection_metadata={"hnsw:space":"cosine"},persist_directory="/content/paris_db")

In [ ]:
#load the vector stores

load_un_sdg_db = Chroma(persist_directory="/content/un_sdg_db",embedding_function=hf)
load_paris_db = Chroma(persist_directory="/content/paris_db",embedding_function=hf)

In [ ]:
#init merger retriever and perform semantic search

retriever_un_sdg = load_un_sdg_db.as_retriever(search_type="similarity",search_kwargs={"k":2,"include_metadata":True})
retriever_paris = load_paris_db.as_retriever(search_type="similarity",search_kwargs={"k":2,"include_metadata":True})

In [ ]:
lotr = MergerRetriever(retrievers=[retriever_un_sdg,retriever_paris])


In [ ]:
query = "Explain the RAG architecture?"

In [ ]:
res = lotr.get_relevant_documents(query)


In [ ]:
#Long Context Reorder(Lost in the middle problem Fix)
reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(res)